In [1]:
import numpy as np
import pandas as pd

In [2]:
import transformers

In [3]:
def calculate_f1_score(actual_keyphrases, predicted_keyphrases):
    actual_set = set([s.lower() for s in actual_keyphrases])
    predicted_set = set(s.lower() for s in predicted_keyphrases)

    true_positives = len(actual_set.intersection(predicted_set))
    precision = true_positives / len(predicted_set) if predicted_set else 0
    recall = true_positives / len(actual_set) if actual_set else 0

    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
    # print(f"Precision: {precision} \t Reacll: {recall} \t F1 Score: {f1_score}")
    return precision, recall, f1_score


actual_keyphrases = ['keyphrase one', 'keyphrase two', 'keyphrase three']
predicted_keyphrases = ['keyphrase one', 'keyphrase four', 'keyphrase two']
p,r,f1_score = calculate_f1_score(actual_keyphrases, predicted_keyphrases)
print(f"F1 Score: {f1_score}")

F1 Score: 0.6666666666666666


In [4]:
def convert_to_unigrams(keyphrases):
    """Convert a list of keyphrases to a set of unigrams."""
    return set(unigram for keyphrase in keyphrases for unigram in keyphrase.split())

def partial_f1_score(gold_keyphrases, extracted_keyphrases):
    """Calculate the F1 score for partial matches between two lists of keyphrases."""
    gold_unigrams = convert_to_unigrams(gold_keyphrases)
    extracted_unigrams = convert_to_unigrams(extracted_keyphrases)

    true_positives = len(gold_unigrams.intersection(extracted_unigrams))
    false_positives = len(extracted_unigrams - gold_unigrams)
    false_negatives = len(gold_unigrams - extracted_unigrams)

    if true_positives == 0:
        return 0,0,0  # Return 0 to avoid division by zero if there are no true positives

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    return precision, recall, f1_score

# Example usage
gold_keyphrases = ['machine learning', 'deep learning', 'neural networks']
extracted_keyphrases = ['learning in machines', 'networks', 'deep neural learning']

precision,recall,f1_score = partial_f1_score(gold_keyphrases, extracted_keyphrases)
# print(f"F1 Score for partial match: {f1_score}")
print(f"Precision: {precision} \t Reacll: {recall} \t F1 Score for partial match: {f1_score}")

Precision: 0.6666666666666666 	 Reacll: 0.8 	 F1 Score for partial match: 0.7272727272727272


In [11]:
from datasets import load_dataset

dataset = load_dataset("taln-ls2n/inspec")

/Users/nishanthkrishna/miniforge3/envs/nlp/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for taln-ls2n/inspec contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/taln-ls2n/inspec
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [12]:
df = pd.DataFrame(dataset['train'])

In [13]:
test_df = pd.DataFrame(dataset['test'])
validation_df = pd.DataFrame(dataset['validation'])

In [14]:
def lowercase(my_list):
    return [i.lower() for i in my_list]

In [15]:
df['abstract'] = df['abstract'].apply(lambda x: x.lower())
df['keyphrases'] = df['keyphrases'].apply(lowercase)

In [16]:
df

,id,title,abstract,keyphrases,prmu
0,761,Towards a NMR implementation of a quantum latt...,recent theoretical results suggest that an arr...,"[nmr implementation, quantum lattice gas algor...","[P, P, P, P, P, P, P, M]"
1,724,Banking on SMA funds [separately managed accou...,from investment management to technology to ba...,"[separately managed accounts, investment manag...","[P, P, P, P, P, P]"
2,1371,Design methodology for diagnostic strategies f...,this paper presents a method for the construct...,"[design methodology, diagnostic strategies, in...","[P, P, P, P, P]"
3,1334,A shy invariant of graphs,moving from a well known result of p.l. hammer...,"[graph invariant, induced odd cycles, minimum ...","[P, P, P, P, P, P]"
4,1419,PacketVideo. One step ahead of the streaming w...,"go beyond the hype, however, and it's clear th...","[packetvideo, wireless devices, mpeg-4, wirele...","[P, P, P, P, P, P, P, R]"
...,...,...,...,...,...
995,1124,Data extraction from the Web based on pre-defi...,"with the development of the internet, the worl...","[data extraction, schema, internet, informatio...","[P, P, P, P, P, R, M, U, U]"
996,118,Sensorless control of induction motor drives,controlled induction motor drives without mech...,"[sensorless control, induction motor drives, r...","[P, P, P, P, P, P, P, P, P, P, P, P, P, P, P, ..."
997,635,Detection and estimation of abrupt changes in ...,detection of change-points in normal means is ...,"[generalized likelihood ratio test statistic, ...","[P, P, P, P, P, P, P, P, P, P, P, P, P, P, P, ..."
998,1260,A dataflow computer which accelerates executio...,"in the dataflow machine, it is important to av...","[dataflow computer, sequential programs, prece...","[P, P, P, P, P, P, P, P, P, R, M, M]"


In [17]:
# Model parameters
from transformers import (
    Text2TextGenerationPipeline,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)


class KeyphraseGenerationPipeline(Text2TextGenerationPipeline):
    def __init__(self, model, keyphrase_sep_token=";", *args, **kwargs):
        super().__init__(
            model=AutoModelForSeq2SeqLM.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )
        self.keyphrase_sep_token = keyphrase_sep_token

    def postprocess(self, model_outputs):
        results = super().postprocess(
            model_outputs=model_outputs
        )
        return [[keyphrase.strip() for keyphrase in result.get("generated_text").split(self.keyphrase_sep_token) if keyphrase != ""] for result in results]


In [18]:
# Load pipeline
model_name = "ml6team/keyphrase-generation-t5-small-inspec"
generator = KeyphraseGenerationPipeline(model=model_name)

In [19]:
text = df['abstract'][0]

keyphrases = generator(text)

print(keyphrases)

[['quantum information processors', 'fluid dynamics problems', 'quantum lattice-gas algorithms']]


/Users/nishanthkrishna/miniforge3/envs/nlp/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [20]:
import spacy
from keybert import KeyBERT

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Sample text
# text = "The quick brown fox jumps over the lazy dog. It barked and played in the garden."

# Process the text with spaCy for POS tagging
doc = nlp(text)

# Filter words based on POS tags
# categories = ['NOUN', 'PROPN', 'VERB', 'ADJ']
# filtered_text = " ".join([token.text for token in doc if token.pos_ in categories])

# Now, use KeyBERT to extract keywords from the filtered text
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=100)

# Print all extracted keywords and their scores
all_words = [word for word,_ in keywords if _>0]
keywords_score_dict = {}
for word, score in keywords:
    if score>0:
        print(f"{word}: {score}")
        keywords_score_dict[word]=score

qubit: 0.3605
quantum: 0.3381
nmr: 0.3305
lattice: 0.2493
diffusion: 0.2144
processors: 0.2031
dynamics: 0.2009
simulation: 0.187
density: 0.1866
ensemble: 0.1732
gas: 0.1584
algorithms: 0.153
implementation: 0.1522
pulse: 0.1501
magnetic: 0.1438
architectures: 0.142
qlga: 0.1381
resonance: 0.1371
algorithm: 0.1349
steps: 0.1305
encode: 0.1301
nonlinear: 0.1273
through: 0.1246
fluid: 0.1196
channels: 0.1072
via: 0.0968
nuclear: 0.0946
initial: 0.0865
communicating: 0.0802
burgers: 0.0801
classical: 0.0785
information: 0.0746
array: 0.0745
follow: 0.0654
techniques: 0.0615
first: 0.061
theoretical: 0.058
progress: 0.0565
qualitatively: 0.0538
running: 0.0533
been: 0.0529
recent: 0.05
have: 0.0479
observed: 0.0463
in: 0.046
equations: 0.0453
solves: 0.0452
an: 0.0447
mass: 0.0321
then: 0.0262
the: 0.0233
describe: 0.0229
evolved: 0.0168
improved: 0.0163
two: 0.0163
methods: 0.0126
suggest: 0.0104
shown: 0.0099
manipulated: 0.0071
experimental: 0.0067
on: 0.0052
control: 0.0046


In [21]:
sorted(all_words)

['algorithm',
 'algorithms',
 'an',
 'architectures',
 'array',
 'been',
 'burgers',
 'channels',
 'classical',
 'communicating',
 'control',
 'density',
 'describe',
 'diffusion',
 'dynamics',
 'encode',
 'ensemble',
 'equations',
 'evolved',
 'experimental',
 'first',
 'fluid',
 'follow',
 'gas',
 'have',
 'implementation',
 'improved',
 'in',
 'information',
 'initial',
 'lattice',
 'magnetic',
 'manipulated',
 'mass',
 'methods',
 'nmr',
 'nonlinear',
 'nuclear',
 'observed',
 'on',
 'processors',
 'progress',
 'pulse',
 'qlga',
 'qualitatively',
 'quantum',
 'qubit',
 'recent',
 'resonance',
 'running',
 'shown',
 'simulation',
 'solves',
 'steps',
 'suggest',
 'techniques',
 'the',
 'then',
 'theoretical',
 'through',
 'two',
 'via']

In [22]:
true_values = []
for i in df['keyphrases'][0]:
    true_values += i.split(" ")
true_values = list(set(true_values))
true_values

['nuclear',
 'nonlinear',
 'quantum',
 'resonance',
 'diffusion',
 'information',
 'nmr',
 'dynamics',
 'algorithm',
 'two-qubit',
 'processors',
 'fluid',
 'lattice',
 'information.processors',
 'magnetic',
 'problems',
 'burgers',
 'equations',
 'implementation',
 'gas',
 'equation']

In [23]:
calculate_f1_score(true_values, all_words)

(0.27419354838709675, 0.8095238095238095, 0.4096385542168674)

In [24]:
def get_sequential_tokens(doc, interested_pos, lower):
    # to get sequential tokens from the text
    sentences = []
    for sent in doc.sents:
        selected_words = []
        for token in sent:
            if token.pos_ in interested_pos:
                if lower:
                    selected_words.append(token.text.lower())
                else:
                    selected_words.append(token.text)
            else:
                selected_words.append("*")
        sentences.append(selected_words)
    return sentences

def get_n_grams(sentences):
    # to get n_grams outta sequential tokens
    n_grams = []
    for sent in sentences:
        temp_list = []
        for word in sent:
            if word == '*':
                if len(temp_list)!=0:
                    n_grams.append(temp_list)
                    temp_list = []
            else:
                temp_list.append(word)
    return n_grams

In [25]:
def find_longest_sequence_and_scores(n_grams, unigrams, keywords_score_dict):
    longest_sequences = []
    
    for n_gram in n_grams:
        current_sequence = []
        longest_sequence = []
        
        for word in n_gram:
            if word in unigrams:
                current_sequence.append(word)
                if len(current_sequence) > len(longest_sequence):
                    longest_sequence = current_sequence.copy()
            else:
                current_sequence = []
        
        if len(longest_sequence)>0:
            longest_sequences.append(longest_sequence)
    
    longest_sequences = [' '.join(i) for i in sorted(longest_sequences, key=len, reverse=True)]

    final_scores = {}
    for seq in longest_sequences:
        score = 0
        for word in seq.split(" "):
            score += keywords_score_dict[word]
        final_scores[seq] = score
    
    return longest_sequences, final_scores

In [26]:
def lemmed(my_list):
    lemmed_list = []
    for seq in my_list:
        lemmed_list.append(' '.join([token.lemma_ for token in nlp(seq)]))
    return lemmed_list

In [27]:
interested_pos = ['NOUN', 'PROPN', 'VERB', 'ADJ']

In [28]:
doc = nlp(df['abstract'][0].replace('-',' '))

In [29]:
doc

recent theoretical results suggest that an array of quantum information processors communicating via classical channels can be used to solve fluid dynamics problems. quantum lattice gas algorithms (qlga) running on such architectures have been shown to solve the diffusion equation and the nonlinear burgers equations. in this report, we describe progress towards an ensemble nuclear magnetic resonance (nmr) implementation of a qlga that solves the diffusion equation. the methods rely on nmr techniques to encode an initial mass density into an ensemble of two qubit quantum information processors. using standard pulse techniques, the mass density can then manipulated and evolved through the steps of the algorithm. we provide the experimental results of our first attempt to realize the nmr implementation. the results qualitatively follow the ideal simulation, but the observed implementation errors highlight the need for improved control

In [30]:
sents = get_sequential_tokens(doc, interested_pos, lower=False)
n_grams = get_n_grams(sents)

In [31]:
n_grams

[['recent', 'theoretical', 'results', 'suggest'],
 ['array'],
 ['quantum', 'information', 'processors', 'communicating'],
 ['classical', 'channels'],
 ['used'],
 ['solve', 'fluid', 'dynamics', 'problems'],
 ['quantum', 'lattice', 'gas', 'algorithms'],
 ['qlga'],
 ['running'],
 ['such', 'architectures'],
 ['shown'],
 ['solve'],
 ['diffusion', 'equation'],
 ['nonlinear', 'burgers', 'equations'],
 ['report'],
 ['describe', 'progress'],
 ['ensemble', 'nuclear', 'magnetic', 'resonance'],
 ['nmr'],
 ['implementation'],
 ['qlga'],
 ['solves'],
 ['diffusion', 'equation'],
 ['methods', 'rely'],
 ['nmr', 'techniques'],
 ['encode'],
 ['initial', 'mass', 'density'],
 ['ensemble'],
 ['qubit', 'quantum', 'information', 'processors'],
 ['using', 'standard', 'pulse', 'techniques'],
 ['mass', 'density'],
 ['manipulated'],
 ['evolved'],
 ['steps'],
 ['algorithm'],
 ['provide'],
 ['experimental', 'results'],
 ['first', 'attempt'],
 ['realize'],
 ['nmr', 'implementation'],
 ['results'],
 ['follow'],
 ['id

In [32]:
sorted(all_words)

['algorithm',
 'algorithms',
 'an',
 'architectures',
 'array',
 'been',
 'burgers',
 'channels',
 'classical',
 'communicating',
 'control',
 'density',
 'describe',
 'diffusion',
 'dynamics',
 'encode',
 'ensemble',
 'equations',
 'evolved',
 'experimental',
 'first',
 'fluid',
 'follow',
 'gas',
 'have',
 'implementation',
 'improved',
 'in',
 'information',
 'initial',
 'lattice',
 'magnetic',
 'manipulated',
 'mass',
 'methods',
 'nmr',
 'nonlinear',
 'nuclear',
 'observed',
 'on',
 'processors',
 'progress',
 'pulse',
 'qlga',
 'qualitatively',
 'quantum',
 'qubit',
 'recent',
 'resonance',
 'running',
 'shown',
 'simulation',
 'solves',
 'steps',
 'suggest',
 'techniques',
 'the',
 'then',
 'theoretical',
 'through',
 'two',
 'via']

In [33]:
res, scores = find_longest_sequence_and_scores(n_grams, all_words, keywords_score_dict)
res

['quantum information processors communicating',
 'quantum lattice gas algorithms',
 'ensemble nuclear magnetic resonance',
 'qubit quantum information processors',
 'nonlinear burgers equations',
 'initial mass density',
 'recent theoretical',
 'classical channels',
 'fluid dynamics',
 'describe progress',
 'nmr techniques',
 'pulse techniques',
 'mass density',
 'nmr implementation',
 'observed implementation',
 'array',
 'qlga',
 'running',
 'architectures',
 'shown',
 'diffusion',
 'nmr',
 'implementation',
 'qlga',
 'solves',
 'diffusion',
 'methods',
 'encode',
 'ensemble',
 'manipulated',
 'evolved',
 'steps',
 'algorithm',
 'experimental',
 'first',
 'follow',
 'simulation']

In [34]:
scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
scores

{'qubit quantum information processors': 0.9763,
 'quantum lattice gas algorithms': 0.8988,
 'quantum information processors communicating': 0.696,
 'ensemble nuclear magnetic resonance': 0.5487,
 'nmr implementation': 0.4827,
 'nmr techniques': 0.392,
 'nmr': 0.3305,
 'fluid dynamics': 0.3205,
 'initial mass density': 0.30519999999999997,
 'nonlinear burgers equations': 0.2527,
 'mass density': 0.21869999999999998,
 'diffusion': 0.2144,
 'pulse techniques': 0.2116,
 'observed implementation': 0.1985,
 'simulation': 0.187,
 'classical channels': 0.1857,
 'ensemble': 0.1732,
 'implementation': 0.1522,
 'architectures': 0.142,
 'qlga': 0.1381,
 'algorithm': 0.1349,
 'steps': 0.1305,
 'encode': 0.1301,
 'recent theoretical': 0.10800000000000001,
 'describe progress': 0.0794,
 'array': 0.0745,
 'follow': 0.0654,
 'first': 0.061,
 'running': 0.0533,
 'solves': 0.0452,
 'evolved': 0.0168,
 'methods': 0.0126,
 'shown': 0.0099,
 'manipulated': 0.0071,
 'experimental': 0.0067}

In [35]:
final_res = list(scores.keys())[:10]

In [36]:
final_res

['qubit quantum information processors',
 'quantum lattice gas algorithms',
 'quantum information processors communicating',
 'ensemble nuclear magnetic resonance',
 'nmr implementation',
 'nmr techniques',
 'nmr',
 'fluid dynamics',
 'initial mass density',
 'nonlinear burgers equations']

In [37]:
df['keyphrases'][0]

['nmr implementation',
 'quantum lattice gas algorithm',
 'quantum information processors',
 'fluid dynamics problems',
 'diffusion equation',
 'nonlinear burgers equations',
 'nuclear magnetic resonance',
 'two-qubit quantum information.processors']

In [38]:
calculate_f1_score(lemmed(df['keyphrases'][0]),lemmed(final_res) )

(0.3, 0.375, 0.33333333333333326)

In [39]:
partial_f1_score(lemmed(df['keyphrases'][0]),lemmed(final_res) )

(0.7391304347826086, 0.7727272727272727, 0.7555555555555555)

In [40]:
df

,id,title,abstract,keyphrases,prmu
0,761,Towards a NMR implementation of a quantum latt...,recent theoretical results suggest that an arr...,"[nmr implementation, quantum lattice gas algor...","[P, P, P, P, P, P, P, M]"
1,724,Banking on SMA funds [separately managed accou...,from investment management to technology to ba...,"[separately managed accounts, investment manag...","[P, P, P, P, P, P]"
2,1371,Design methodology for diagnostic strategies f...,this paper presents a method for the construct...,"[design methodology, diagnostic strategies, in...","[P, P, P, P, P]"
3,1334,A shy invariant of graphs,moving from a well known result of p.l. hammer...,"[graph invariant, induced odd cycles, minimum ...","[P, P, P, P, P, P]"
4,1419,PacketVideo. One step ahead of the streaming w...,"go beyond the hype, however, and it's clear th...","[packetvideo, wireless devices, mpeg-4, wirele...","[P, P, P, P, P, P, P, R]"
...,...,...,...,...,...
995,1124,Data extraction from the Web based on pre-defi...,"with the development of the internet, the worl...","[data extraction, schema, internet, informatio...","[P, P, P, P, P, R, M, U, U]"
996,118,Sensorless control of induction motor drives,controlled induction motor drives without mech...,"[sensorless control, induction motor drives, r...","[P, P, P, P, P, P, P, P, P, P, P, P, P, P, P, ..."
997,635,Detection and estimation of abrupt changes in ...,detection of change-points in normal means is ...,"[generalized likelihood ratio test statistic, ...","[P, P, P, P, P, P, P, P, P, P, P, P, P, P, P, ..."
998,1260,A dataflow computer which accelerates executio...,"in the dataflow machine, it is important to av...","[dataflow computer, sequential programs, prece...","[P, P, P, P, P, P, P, P, P, R, M, M]"


In [41]:
def extract(text, number=20):
    def get_sequential_tokens(doc, interested_pos, lower):
        # to get sequential tokens from the text
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                if token.pos_ in interested_pos:
                    if lower:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
                else:
                    selected_words.append("*")
            sentences.append(selected_words)
        return sentences
    
    def get_n_grams(sentences):
        # to get n_grams outta sequential tokens
        n_grams = []
        for sent in sentences:
            temp_list = []
            for word in sent:
                if word == '*':
                    if len(temp_list)!=0:
                        n_grams.append(temp_list)
                        temp_list = []
                else:
                    temp_list.append(word)
        return n_grams
        
    def find_longest_sequence_and_scores(n_grams, unigrams, keywords_score_dict):
        longest_sequences = []
        
        for n_gram in n_grams:
            current_sequence = []
            longest_sequence = []
            
            for word in n_gram:
                if word in unigrams:
                    current_sequence.append(word)
                    if len(current_sequence) > len(longest_sequence):
                        longest_sequence = current_sequence.copy()
                else:
                    current_sequence = []
            
            if len(longest_sequence)>0:
                longest_sequences.append(longest_sequence)
        
        longest_sequences = [' '.join(i) for i in sorted(longest_sequences, key=len, reverse=True)]
    
        final_scores = {}
        for seq in longest_sequences:
            score = 0
            for word in seq.split(" "):
                score += keywords_score_dict[word]
            final_scores[seq] = score
        
        return longest_sequences, final_scores
    
    def lemmed(my_list):
        lemmed_list = []
        for seq in my_list:
            lemmed_list.append(' '.join([token.lemma_ for token in nlp(seq)]))
        return lemmed_list

    # ------------------------------------------------------------------------------------------------------------------------
    
    nlp = spacy.load("en_core_web_sm")
    spacy_text = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in spacy_text])
    doc = nlp(spacy_text)
    
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=100)
    
    all_words = [word for word,_ in keywords if _>0]
    keywords_score_dict = {}
    for word, score in keywords:
        if score>0:
            # print(f"{word}: {score}")
            keywords_score_dict[word]=score
    
    interested_pos = ['NOUN', 'PROPN', 'VERB', 'ADJ']
    sents = get_sequential_tokens(doc, interested_pos, lower=False)
    n_grams = get_n_grams(sents)

    res, scores = find_longest_sequence_and_scores(n_grams, all_words, keywords_score_dict)
    scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
    
    final_res = list(scores.keys())[:number]

    return lemmed(final_res)

In [42]:
# import pandas as pd
# from tqdm.auto import tqdm
# tqdm.pandas()

# # Assuming 'lemmed' and 'extract' are your functions for lemmatization and keyphrase extraction.
# # And assuming 'df' is your DataFrame with columns 'keyphrases' and 'abstract'.

# # Apply the 'lemmed' function to the 'keyphrases' column with a progress bar
# df['true'] = df['keyphrases'].progress_apply(lemmed)

# # Apply the 'extract' function to the 'abstract' column with a progress bar
# df['preds'] = df['abstract'].progress_apply(extract)

In [43]:
df = pd.read_csv("t5_2.csv")

In [44]:
pred = list(df['preds'])
true_values = list(df['true'])

p = 0
r = 0
total_f1 = 0
partial_f1 = 0
for y,y_hat in zip(true_values, pred):
    total_f1 += calculate_f1_score(y,y_hat)
    partial_f1 += partial_f1_score(y,y_hat)
print(f"Exact F1: {total_f1/len(pred)}")
print(f"Partial F1:{partial_f1/len(pred)}")

TypeError: unsupported operand type(s) for +=: 'int' and 'tuple'

In [42]:
df.to_csv("t5_2.csv")

In [168]:
def get_scores(df,true,preds,n=10):
    pred = list(df[f"{preds}"])
    true_values = list(df[f"{true}"])
    
    exact_precision = 0
    exact_recall = 0
    exact_f1 = 0
    partial_precision = 0
    partial_recall = 0
    partial_f1 = 0
    i=0
    for y,y_hat in zip(true_values, pred):
        e_p,e_r,e_f1 = calculate_f1_score(y,y_hat[:n])
        exact_f1 += e_f1
        exact_precision += e_p
        exact_recall += e_r
        p_p,p_r,p_f1 = partial_f1_score(y,y_hat[:n])
        partial_f1 += p_f1
        partial_precision += p_p
        partial_recall += p_r
    print("---"*40)
    print(f"Precision: {exact_precision/len(pred)} \t Reacll: {exact_recall/len(pred)} \t Exact F1: {exact_f1/len(pred)}")
    print("---"*40)
    print(f"Precision: {partial_precision/len(pred)} \t Reacll: {partial_recall/len(pred)} \t Partial F1: {partial_f1/len(pred)}")
    print("---"*40)

In [157]:
get_scores(df,'true','preds',5)

------------------------------------------------------------------------------------------------------------------------
Exact F1: 0.21165691647273954 	 Precision: 0.3045999999999983 	 Reacll: 0.17768604624800505
------------------------------------------------------------------------------------------------------------------------
Partial F1: 0.46394135121348384 	 Precision: 0.660525524475525 	 Reacll: 0.3944366412550005
------------------------------------------------------------------------------------------------------------------------


In [158]:
get_scores(df,'true','preds',10)

------------------------------------------------------------------------------------------------------------------------
Exact F1: 0.2614753087425845 	 Precision: 0.26528571428571424 	 Reacll: 0.2865150920654644
------------------------------------------------------------------------------------------------------------------------
Partial F1: 0.5376599091851008 	 Precision: 0.5924233433273641 	 Reacll: 0.5481750861655554
------------------------------------------------------------------------------------------------------------------------


In [159]:
get_scores(df,'true','preds',20)

------------------------------------------------------------------------------------------------------------------------
Exact F1: 0.2430312919754198 	 Precision: 0.19280114803067638 	 Reacll: 0.36796760938086626
------------------------------------------------------------------------------------------------------------------------
Partial F1: 0.5406408323023981 	 Precision: 0.49088944213127433 	 Reacll: 0.6712053586286151
------------------------------------------------------------------------------------------------------------------------


In [154]:
df['true'][0]

['nmr implementation',
 'quantum lattice gas algorithm',
 'quantum information processor',
 'fluid dynamic problem',
 'diffusion equation',
 'nonlinear burger equation',
 'nuclear magnetic resonance',
 'two - qubit quantum information.processor']

In [45]:
df['preds'][0]

['qubit quantum information processor',
 'quantum information processor communicate',
 'quantum lattice',
 'ensemble nuclear magnetic resonance',
 'nmr implementation',
 'nmr technique',
 'nmr',
 'fluid dynamic',
 'gas algorithm',
 'initial mass density']

In [243]:
calculate_f1_score(df['true'][0], df['preds'][0])

Precision: 0.1 	 Reacll: 0.125 	 F1 Score: 0.11111111111111112


0.11111111111111112

In [163]:
test_df

,id,title,abstract,keyphrases,prmu
0,2007,The creation of a high-fidelity finite element...,A detailed finite element model of the human k...,"[high-fidelity finite element model, kidney, t...","[P, P, P, P, P, P, P, P, P, P, P, P, P, P, P, ..."
1,2042,Hybrid simulation of space plasmas: models wit...,"For pt.III. see Prikl. Mat. Informatika, MAKS ...","[hybrid simulation, space plasmas, massless fl...","[P, P, P, P, P, P, P, P, P, P, P, P, P]"
2,308,On-line Homework/Quiz/Exam applet: freely avai...,The Homework/Quiz/Exam applet is a freely avai...,"[freely available Java software, database conn...","[P, P, P, P, P, P, P, P, P, M, M, R, R, R, R, ..."
3,215,A conceptual framework for evaluation of infor...,The decision to acquire a new information tech...,"[information technology investments, technolog...","[P, P, P, P, P, P, P, P, P, P, P, P, P, P, P, ..."
4,250,Aim for the enterprise: Microsoft Project 2002,"A long-time favorite of project managers, Micr...","[Microsoft Project 2002, Web-based collaborati...","[P, P, P, P, P, P, R]"
...,...,...,...,...,...
495,341,How should team captains order golfers on the ...,I used game theory to examine how team captain...,"[game theory, slate, golf, golfer ordering, Ry...","[P, P, U, R, R]"
496,219,Firewall card shields data,The SlotShield 3000 firewall on a PCI card sav...,"[SlotShield 3000 firewall, PCI card, security,...","[P, P, P, P]"
497,1967,Modeling daily realized futures volatility wit...,"Using singular spectrum analysis (SSA), we mod...","[daily realized futures volatility, singular s...","[P, P, P, P, P, P, P, P, P, P, M, U, M]"
498,2116,Optimization of the characteristics of computa...,The scalableness of resources is taken to mean...,"[computational processes, scalable resources, ...","[P, P, P, P, P, P, P, P, P, P, M]"


In [164]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

# Assuming 'lemmed' and 'extract' are your functions for lemmatization and keyphrase extraction.
# And assuming 'df' is your DataFrame with columns 'keyphrases' and 'abstract'.

# Apply the 'lemmed' function to the 'keyphrases' column with a progress bar
test_df['true'] = test_df['keyphrases'].progress_apply(lemmed)

# Apply the 'extract' function to the 'abstract' column with a progress bar
test_df['preds'] = test_df['abstract'].progress_apply(extract)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [169]:
get_scores(test_df,'true','preds',5)

------------------------------------------------------------------------------------------------------------------------
Precision: 0.2915000000000005 	 Reacll: 0.171508232936406 	 Exact F1: 0.2038569082167069
------------------------------------------------------------------------------------------------------------------------
Precision: 0.6616756127933773 	 Reacll: 0.3735623698033408 	 Partial F1: 0.45356797063765153
------------------------------------------------------------------------------------------------------------------------


In [170]:
get_scores(test_df,'true','preds',10)

------------------------------------------------------------------------------------------------------------------------
Precision: 0.2357793650793651 	 Reacll: 0.2552031711454316 	 Exact F1: 0.23165449803317084
------------------------------------------------------------------------------------------------------------------------
Precision: 0.5804797613185791 	 Reacll: 0.509689632671014 	 Partial F1: 0.5165817661337387
------------------------------------------------------------------------------------------------------------------------


In [171]:
get_scores(test_df,'true','preds',20)

------------------------------------------------------------------------------------------------------------------------
Precision: 0.17458552668521732 	 Reacll: 0.3205708465705938 	 Exact F1: 0.2153073611086031
------------------------------------------------------------------------------------------------------------------------
Precision: 0.485167324044939 	 Reacll: 0.6177772903984373 	 Partial F1: 0.5181069757254992
------------------------------------------------------------------------------------------------------------------------
